In [ ]:
 def wrapper_without_frequency(experimentname,sequencetoimplement,resonant_freq=2.89e9,averages=10,xMin=1.49,yMin=0.61,samples=1000,comment='',start=0,stop=500,steps=500,spacing="lin",lasChan=1,countGateChan=5,countAcquireChan=7,lasInit=5e3,lasRead=5e3,microwaveontime=3e2,use_Ichan=True,buffer1=None,**kwargs):
     numberofpoints=samples*2 #because there are two readouts in single scan
     pixel=numberofpoints*steps
     ni_device.reset_device()
     ps.reset()
     galvo_move(xMin,yMin)
     print("creating sequence")
     # Counter
     CountWidth = nidaqmx.Task()
     ciChannel = CountWidth.ci_channels.add_ci_count_edges_chan('/Dev1/ctr1',edge=nidaqmx.constants.Edge.RISING, initial_count=0, count_direction=nidaqmx.constants.CountDirection.COUNT_UP)
     CountWidth.triggers.pause_trigger.dig_lvl_src='/Dev1/PFI4'
     CountWidth.triggers.pause_trigger.trig_type=nidaqmx.constants.TriggerType.DIGITAL_LEVEL
     CountWidth.triggers.pause_trigger.dig_lvl_when=nidaqmx.constants.Level.LOW
     #CountWidth.timing.cfg_implicit_timing(sample_mode = nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=(pixel)*averages)#samps per channel defines the buffer size for the memory
     CountWidth.timing.cfg_samp_clk_timing(rate=1e8,source='/Dev1/PFI5',active_edge=nidaqmx.constants.Edge.FALLING,sample_mode = nidaqmx.constants.AcquisitionType.FINITE, samps_per_chan=(pixel) )
     cps = []
     callback=[]
     #countsdataset = pd.DataFrame([0],columns=['A'],dtype= object)
     #PUlse streamer gating
     # Digital output
     DigChannel = 'Dev1/port0/line7' #connect this to PFI 4 #this is ctr 1 gate
     DigTask = nidaqmx.Task()
     DigTask.do_channels.add_do_chan(lines = DigChannel)
     DigChannel = 'Dev1/port0/line7' #Defining the port for taking the output
     def readBuffer(task_handle, every_n_samples_event_type, number_of_samples, callback_data):
         CountWidth.in_stream.read_all_avail_samp = True
         readPixels=readerWidth.read_many_sample_uint32(highCount, number_of_samples_per_channel=- 1, timeout=10.0)
         cps.extend(highCount)
         callback.extend([1])
         return 0
     buffersamplecount=numberofpoints
     # Counter read task
     readerWidth = nidaqmx.stream_readers.CounterReader(CountWidth.in_stream)
     highCount = np.zeros(buffersamplecount, dtype = np.uint32)
     #lowCount =  np.zeros(buffersamplecount,dtype = np.uint32)
     source_0.frequency(resonant_freq)
     time.sleep(0.01)
     print("source frequency set to",source_0.frequency())
     # Read after filling the buffer with given number of samples
     CountWidth.register_every_n_samples_acquired_into_buffer_event(buffersamplecount,readBuffer) #after every pixel it will trigger the callback
     # Start tasks (digital output will be triggered by analog output)
     print("starting DAQ")
     #source_0.on()
     #Adding infinite loop
     t=0
     run=0
     data=[]
     finaldata=[]
     print("Preparing for the experiment")
     print("callback number in beginning:",len(callback))
     for i in trange(averages):
         print(run)
         time.sleep(0.05)
         #CountWidth.control(nidaqmx.constants.TaskMode.TASK_RESERVE)
         time.sleep(0.05)
         CountWidth.start()
         sequence=sequencetoimplement(start=start,stop=stop,steps=steps,lasChan=lasChan,countGateChan=countGateChan,countAcquireChan=countAcquireChan,lasInit=lasInit,gateLen=lasRead,micrPulse=microwaveontime,**kwargs)
         ps.setTrigger(start=psl.TriggerStart.HARDWARE_RISING,rearm=psl.TriggerRearm.AUTO)
         seq_num=0
         time_start=time.time()
         for s in sequence:
             t1=len(callback)
             #s.plot()
             seq_num=seq_num+1
             ps.stream(s, n_runs=samples)
             #print(seq_num)
             time_start=time.time()
             DigTask.write(True)
             while len(callback)==t1:
                 time.sleep(0.005)
             time_end=time.time()
             DigTask.write(False)
         print('time taken for on stream of all sequences', time_end-time_start)
         print("callback number average end:",len(callback))
         run=run+1
         CountWidth.control(nidaqmx.constants.TaskMode.TASK_UNRESERVE)
     #data.append(np.mean(temp))
     print(seq_num)
     source_0.off()
     data=datainterpretdaq_pulsed_unreserve(cps,pixel)
     #
     save_data_general(list=data,integration_window=lasRead,name_of_experiment=experimentname,xminsave=xMin,xmaxsave=xMin,yminsave=yMin,ymaxsave=yMin,comment='')
     CountWidth.close()
     DigTask.close()
     return data